In [ ]:
import weaviate
import os
from weaviate import classes as wvc
client = weaviate.connect_to_weaviate_cloud(
    cluster_url="",
    auth_credentials=weaviate.auth.AuthApiKey(""),
    headers={
        "X-Openai-Api-Key": os.getenv("OPENAI_API_KEY")
    }
)
# client = weaviate.connect_to_local(
#     headers={
#         "X-Openai-Api-Key": os.getenv("OPENAI_API_KEY")
#     }
# )

In [ ]:
from weaviate import classes as wvc
#client.collections.delete("Index")
client.collections.create(
    "Index",
    vectorizer_config=wvc.config.Configure.Vectorizer.text2vec_openai(
        model="text-embedding-3-small",
    ),
    generative_config=wvc.config.Configure.Generative.openai()
)

In [31]:
# read csv file
import pandas as pd
from weaviate.util import generate_uuid5
collection = client.collections.get("Index")
df = pd.read_csv("trabalhos_60700009.csv")
df.columns = map(str.lower, df.columns)
# read only 10 objects
df = df.head(500)
with collection.batch.dynamic() as batch:
    for index, row in df.iterrows():
        batch.add_object(
            dict(row),
            uuid=generate_uuid5(row["indice"]),
        )
if collection.batch.failed_objects:
    print(collection.batch.failed_objects)
else:
    print("Imported with success")

Imported with success


In [43]:
collection.aggregate.over_all()

AggregateReturn(properties={}, total_count=500)

In [45]:
query = collection.generate.near_text(
    query="Quais os projetos relacionados a ontologia e Organização do conhecimento?",
    return_metadata=wvc.query.MetadataQuery(distance=True),
    grouped_task="Sumarize os conteúdos e explique porque cada um está próximo da busca: Quais os projetos relacionados a ontologia e Organização do conhecimento?",
    limit=200,
    auto_limit=1
)
for o in query.objects:
    print("###"*10 + str(o.metadata.distance))
    print("RESUMO", o.properties.get("ds_resumo"))
    print("NOME PRODUCAO", o.properties.get("nm_producao"))

print("------"* 10)
print(query.generated)


##############################0.38957417011260986
RESUMO O desenvolvimento científico e tecnológico vem trazendo em seu escopo novas estruturas de.conteúdos. Relações de toda ordem montam teias que ligam dados, informações, pessoas e.locais em um mesmo ambiente não mais físico. A velocidade determinante deste novo século.tem gerado inquietações acerca da nova situação que envolve todas as áreas do conhecimento..Grandes volumes de conteúdos precisam ser recuperados com rapidez e precisão atendendo às.expectativas dos usuários. Sistemas de Recuperação de Informação são desenvolvidos na.tentativa de solucionar esta questão. A pesquisa se insere no universo das linguagens.aplicadas nesses sistemas e sua motivação é fruto da leitura e reflexão de estudos anteriores.sobre o tema, envolvendo aspectos da sua estrutura, conceitos, relacionamentos e variações..Abordando a linguagem natural e suas novas formas de representação, são estudadas as.ontologias de um Sistema de Recuperação de Informaçõ

In [30]:
len(query.objects)

1